In [1]:
import os
import json
import asyncio
from src.rpc_provider import Web3AsyncRouter, AsyncRpcClient, AsyncRpcScheduler, RpcPool, RpcErrorResult
from src.logging import log
from src.web3_utils import current_utctime, normalize_block

# -----------------------------
# 配置
# -----------------------------
RPC_CONFIG_PATH = "/etc/ingestion/rpc_providers.json"

CHAIN = os.getenv("CHAIN", "bsc").lower()
RESUME_FROM_LAST = os.getenv("RESUME_FROM_LAST", "True").lower() in ("1", "true", "yes")

# -----------------------------
# Job Name & Kafka IDs
# -----------------------------
if RESUME_FROM_LAST:
    JOB_NAME = f"{CHAIN}_backfill_resume"        # 固定名，Kafka checkpoint 能被复用
else:
    JOB_NAME = f"{CHAIN}_realtime_{current_utctime()}"  # 每次唯一，从最新block开始

RPC_MAX_TIMEOUT = int(os.getenv("RPC_MAX_TIMEOUT", "10"))
RPC_MAX_INFLIGHT = int(os.getenv("RPC_MAX_INFLIGHT", "5"))  # 并发数量
TRANSACTIONAL_ID = f"blockchain.ingestion.{CHAIN}.{current_utctime()}" # TRANSACTIONAL_ID每次不一样，EOS由Compact State Topic实现

# -----------------------------
# 加载 RPC 配置
# -----------------------------
rpc_configs = json.load(open(RPC_CONFIG_PATH))

# 初始化rpc pool
rpc_pool = RpcPool.from_config(rpc_configs, CHAIN)

{"ts": "2026-01-22T14:50:16.728Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_enabled", "chain": "bsc", "rpc": "quiknode", "key_envs": ["QUIKNODE_KEY"], "weight": 8}
{"ts": "2026-01-22T14:50:16.729Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_enabled", "chain": "bsc", "rpc": "moralis", "key_envs": ["MORALIS_KEY_B"], "weight": 8}
{"ts": "2026-01-22T14:50:16.729Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_enabled", "chain": "bsc", "rpc": "infura", "key_envs": ["INFURA_KEY_A", "INFURA_KEY_B", "INFURA_KEY_C"], "weight": 8}
{"ts": "2026-01-22T14:50:16.730Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_enabled", "chain": "bsc", "rpc": "blockpi", "key_envs": ["BLOCKPI_BSC_KEY_A", "BLOCKPI_BSC_KEY_B", "BLOCKPI_BSC_KEY_C"], "weight": 8}
{"ts": "2026-01-22T14:50:16.731Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_enabled", "chain": "bsc", "rpc": "ankr", "key_envs": ["ANKR_KEY_A", "ANKR_KEY_B", "ANKR_KEY_C"], "weight": 6}
{"

In [3]:
async def fetch_and_push():
    client = AsyncRpcClient(timeout=RPC_MAX_TIMEOUT)
    router = Web3AsyncRouter(rpc_pool, client)
    scheduler = AsyncRpcScheduler(
        router=router,
        max_workers = 1,
        max_inflight=RPC_MAX_INFLIGHT,  #🔥 真正并发 RPC 数 # max_inflight ≈ (provider 数 × key 数) × 0.6
        max_queue=RPC_MAX_INFLIGHT * 5,
    )
    
    tasks = []
    
    for _ in range(20):
        tasks.append(
            asyncio.create_task(
                scheduler.submit("eth_blockNumber", [])
            )
        )
    
    results = await asyncio.gather(*tasks)

    for r in results:
        if isinstance(r, RpcErrorResult):
            log.warning(
                "rpc_failed",
                extra={
                    "provider": r.rpc,
                    "key": r.key_env,
                    "task_id": r.meta.task_id,
                    "worker": r.wid,
                    "error": type(r.error).__name__,
                },
            )
            continue
    
        result, rpc, key_env, trace, _, meta = r
        block_num = normalize_block(result)
    
        log.info(
            "rpc_call_done",
            extra={
                "task_id": meta.task_id,
                "rpc": rpc,
                "key_env": key_env,
                "block": block_num,
                "latency_ms": round(trace.total_ms, 2),
            },
        )

    await scheduler.close()

In [4]:
# worker as dispatcher - V3
await fetch_and_push()

{"ts": "2026-01-22T14:51:25.635Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_dispatcher_started", "worker": 0}
{"ts": "2026-01-22T14:51:25.637Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_dispatch", "task_id": 1, "worker": 0, "method": "eth_blockNumber", "queue_wait_ms": 0.12}
{"ts": "2026-01-22T14:51:25.638Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_dispatch", "task_id": 2, "worker": 0, "method": "eth_blockNumber", "queue_wait_ms": 1.0}
{"ts": "2026-01-22T14:51:25.638Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_dispatch", "task_id": 3, "worker": 0, "method": "eth_blockNumber", "queue_wait_ms": 1.63}
{"ts": "2026-01-22T14:51:25.639Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_dispatch", "task_id": 4, "worker": 0, "method": "eth_blockNumber", "queue_wait_ms": 2.89}
{"ts": "2026-01-22T14:51:25.640Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_dispatch", "task_id": 5, "worker": 0, "method": "eth_bl

In [ ]:
# worker as dispatcher - V2
await fetch_and_push()

In [ ]:
async def fetch_and_push():
    client = AsyncRpcClient(timeout=RPC_MAX_TIMEOUT)
    router = Web3AsyncRouter(rpc_pool, client)
    scheduler = AsyncRpcScheduler(
        router,
        max_workers=MAX_WORKERS, # dispatcher 数量
        max_queue=MAX_RPC_REQ,
        max_inflight=5,  #🔥 真正并发 RPC 数 # max_inflight ≈ (provider 数 × key 数) × 0.6
        

    )
    for _ in range(MAX_RPC_REQ):
        result, rpc, key_env, trace, worker_id, meta = await scheduler.submit(
            "eth_blockNumber", []
        )
        block_num = normalize_block(result)
        # log.info(
        #     "rpc_call_done",
        #     extra={
        #         "task_id": meta.task_id,
        #         "worker": worker_id,
        #         "rpc": rpc,
        #         "key_env": key_env,
        #         "block": block_num,
        #         "latency_ms": round(trace.total_ms, 2),
        #         "dns_ms": round(trace.dns_ms, 2),
        #         "tcp_ms": round(trace.tcp_ms, 2),
        #         "http_wait_ms": round(trace.http_wait_ms, 2)
        #     },
        # )

    await scheduler.close()

In [ ]:
# worker as dispatcher  - V1
await fetch_and_push()

In [ ]:
# worker as rpc executor 
# await fetch_and_push()